# 1. Load Configuration File

In [4]:
from pathlib import Path
from box import Box  
from vast.utils import load_yaml

# Read configuration file
cfg = Box(load_yaml("config.yaml"))

# Access configuration parameters
print(cfg.video_downloader.video_url)

print(cfg.subtitle_generator.model.whisper_size)
print(cfg.subtitle_generator.model.language)

https://www.youtube.com/watch?v=oXjxK_X1U28
small
de


# 2. Download Videos

In [5]:
from vast.video_downloader import download_video

url = cfg.video_downloader.video_url
output_dir = Path(cfg.paths.raw_videos)

videos = download_video(url, output_dir)

print(f"Downloaded: {videos}")
print(f"Video saved to: {output_dir}")
print(videos)


[youtube] Extracting URL: https://www.youtube.com/watch?v=oXjxK_X1U28
[youtube] oXjxK_X1U28: Downloading webpage
[youtube] oXjxK_X1U28: Downloading android sdkless player API JSON
[youtube] oXjxK_X1U28: Downloading tv client config
[youtube] oXjxK_X1U28: Downloading tv player API JSON
[youtube] oXjxK_X1U28: Downloading web safari player API JSON
[youtube] oXjxK_X1U28: Downloading player 19712d96-main


         player = https://www.youtube.com/s/player/19712d96/player_ias.vflset/en_US/base.js
         n = ndwPn1DdW-Kc3GnU ; player = https://www.youtube.com/s/player/19712d96/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] oXjxK_X1U28: Downloading m3u8 information
[info] oXjxK_X1U28: Downloading 1 format(s): 299+140
[download] data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4 has already been downloaded


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Detected video codec: h264
Final file: data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4
Downloaded: data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4
Video saved to: data/raw_videos
data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4


# 3. Generate Subtitles

In [8]:
from vast.subtitle_generator import generate_subtitle

video_path = videos
output_dir = Path(cfg.paths.subtitles)
model = cfg.subtitle_generator.model
print(videos)

generate_subtitle(video_path, output_dir, model)

data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4
------------------: {'whisper_size': 'small', 'language': 'de'}
Loading Whisper model: small
Transcribing audio... (language=de)


/Users/humengqing/Documents/Code/VSCode/VAST/.venv/lib/python3.11/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Subtitle (.srt) created: data/subtitles/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.srt
Transcript (.json) created: data/subtitles/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_subtitles.json


{'srt_path': PosixPath('data/subtitles/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.srt'),
 'json_path': PosixPath('data/subtitles/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_subtitles.json')}

# 4. Extract Keyframes

In [9]:
from vast.keyframe_extractor import extract_keyframes

output_dir = Path(cfg.paths.keyframes)
interval = cfg.keyframe_extractor.interval

keyframes = extract_keyframes(videos, output_dir, interval)

print(f"Number of keyframes extracted: {len(keyframes)}")


Extraction completed: 33 keyframes generated.
Keyframe metadata saved to data/keyframes/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_keyframes.json
Number of keyframes extracted: 33


# 5. Segment Videos

In [6]:
from vast.scene_segmenter import detect_scenes, export_scenes
from pathlib import Path

video_path = Path(cfg["paths"]["raw_videos"]) / "Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4"

keyframes = sorted(Path(cfg["paths"]["keyframes"]).glob("*.jpg"))

output_dir = Path(cfg["paths"]["sections"])
# 1️ 检测场景边界
scenes = detect_scenes(keyframes, interval=60.0, method="ssim", threshold=0.6)

# 2️ 导出分段视频 + 自动保存 scene_segments.json
export_scenes(video_path, scenes, output_dir=output_dir)

Detecting scene boundaries: 100%|██████████| 32/32 [00:03<00:00,  9.95it/s]


Detected 8 scenes.


Exporting scenes:   0%|          | 0/8 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Exporting scenes:  12%|█▎        | 1/8 [00:00<00:03,  2.01it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Exporting scenes:  25%|██▌       | 2/8 [00:00<00:01,  3.74it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, 

Export complete! 8 scenes saved to: data/sections
Scene metadata saved to data/sections/scene_segments.json


[{'scene_id': 0,
  'start': 0.0,
  'end': 660.0,
  'duration': 660.0,
  'video_path': 'data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4',
  'output_file': 'data/sections/scene_000.mp4'},
 {'scene_id': 1,
  'start': 660.0,
  'end': 720.0,
  'duration': 60.0,
  'video_path': 'data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4',
  'output_file': 'data/sections/scene_001.mp4'},
 {'scene_id': 2,
  'start': 720.0,
  'end': 960.0,
  'duration': 240.0,
  'video_path': 'data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4',
  'output_file': 'data/sections/scene_002.mp4'},
 {'scene_id': 3,
  'start': 960.0,
  'end': 1020.0,
  'duration': 60.0,
  'video_path': 'data/raw_videos/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024.mp4',
  'output_file': 'data/sections/scene_003.mp4'},
 {'scene_id': 4,
  'start': 102

# 6. Analyze Scenes

In [7]:
from vast.scene_analyzer import analyze_directory

input_dir = Path(cfg.paths.keyframes)
output_dir = Path(cfg.paths.scene_descriptions)
model_name = cfg.scene_analyzer.model.name

print(model_name)

analyze_directory(input_dir, output_dir, model_name)

print("Scene analysis completed!")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Salesforce/blip-image-captioning-base
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0000.jpg
Loading BLIP model: Salesforce/blip-image-captioning-base ...
BLIP model loaded successfully.
Loading sentiment model: cardiffnlp/twitter-roberta-base-sentiment


Device set to use mps:0


Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0001.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0002.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0003.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0004.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0005.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0006.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0007.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0008.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_0009.jpg
Analyzing: Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus La

# 7. Summarize Video Clips(Texts)

In [8]:
from vast.text_summarizer import summarize_sections

summarize_sections(
    subtitles_json="data/subtitles/Hitzige Debatte über Alkohol-Konsum in Deutschland ｜ Markus Lanz vom 12. März 2024_subtitles.json",
    segments_json="data/sections/scene_segments.json",
    output_json="data/text_analysis/text_summaries.json",
    summarizer_model="facebook/bart-large-cnn",
    language="de"
)


Loaded 475 subtitles and 8 segments
Loading summarization model: ml6team/mt5-small-german-finetune-mlsum


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/humengqing/Documents/Code/VSCode/VAST/.venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Section 0: summarized 1902 words.


Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Section 1: summarized 152 words.


...

Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Section 5: summarized 148 words.


Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Section 6: summarized 2227 words.
Section 7: summarized 161 words.
Summaries saved to data/text_analysis/text_summaries.json


[{'start': 0.0,
  'end': 660.0,
  'text': 'Vanessa Grasnickl ist bei uns und Natalie Stüben sitzt bei uns. Und ich würde zunächst mal gerne Frau Grasnickl fragen, weil es um die Frage geht, wie verhält man sich zu verschiedenen Themen? Gibt gerade große Aufregung und vielleicht können wir ihren aktuellen Tweet mal zum Thema Cannabis-Freikabel zeigen. Ich weiß nicht endlich, warum ich auf Frau Stülm-Media bin, weil Sie sie mal lesen, Herr Lanz. Ich bin Ihr treuester Fan. Selten kriegt ihr mal so viel Stofffrahlhaus und in dem Sinne auch noch doppeldeutig gemeint. Frau Grasnickl, wenn Sie dann so etwas lesen wie diesen Tweet hier, vielleicht können wir da einmal ganz kurz zuspielen. Wenn es nach der Ampel geht, dann darf man mit 30 Gramm Cannabis auf dem Spielplatz Kinder zukiffen. Der Staat hat einen Schutzversprechen vor allem Kindern und Jugendlichen gegenüber und so weiter und so weiter. Die Legalisierung darf nicht kommen. Dann habe ich Sie richtig verstanden, auch in Interviews, we

# 8. Generate Narrations

In [9]:
from vast.narration_generator import generate_narration_from_summaries

generate_narration_from_summaries(
    summaries_json="data/text_analysis/text_summaries.json",
    output_dir="data/audio",
    lang="de"  # 德语语音
)


Generating narration for 8 sections...


Generating narration:  12%|█▎        | 1/8 [00:02<00:17,  2.48s/it]

Saved narration: scene_000.mp3


Generating narration:  25%|██▌       | 2/8 [00:04<00:13,  2.29s/it]

Saved narration: scene_001.mp3


...

Generating narration:  88%|████████▊ | 7/8 [00:12<00:01,  1.70s/it]

Saved narration: scene_006.mp3


Generating narration: 100%|██████████| 8/8 [00:13<00:00,  1.69s/it]

Saved narration: scene_007.mp3
Narration metadata saved to data/audio/narration_metadata.json


[{'start': 0.0,
  'end': 660.0,
  'text': 'Vanessa Grasnickl ist bei uns und Natalie Stüben sitzt bei uns. Und ich würde zunächst mal gerne Frau Grasnickl fragen, weil es um die Frage geht, wie verhält man sich zu verschiedenen Themen? Gibt gerade große Aufregung und vielleicht können wir ihren aktuellen Tweet mal zum Thema Cannabis-Freikabel zeigen. Ich weiß nicht endlich, warum ich auf Frau Stülm-Media bin, weil Sie sie mal lesen, Herr Lanz. Ich bin Ihr treuester Fan. Selten kriegt ihr mal so viel Stofffrahlhaus und in dem Sinne auch noch doppeldeutig gemeint. Frau Grasnickl, wenn Sie dann so etwas lesen wie diesen Tweet hier, vielleicht können wir da einmal ganz kurz zuspielen. Wenn es nach der Ampel geht, dann darf man mit 30 Gramm Cannabis auf dem Spielplatz Kinder zukiffen. Der Staat hat einen Schutzversprechen vor allem Kindern und Jugendlichen gegenüber und so weiter und so weiter. Die Legalisierung darf nicht kommen. Dann habe ich Sie richtig verstanden, auch in Interviews, we

# 9. Generate Sign Language